In [ ]:
!git clone https://github.com/michalfaber/keras_Realtime_Multi-Person_Pose_Estimation.git

Cloning into 'keras_Realtime_Multi-Person_Pose_Estimation'...
remote: Enumerating objects: 223, done.
remote: Total 223 (delta 0), reused 0 (delta 0), pack-reused 223
Receiving objects: 100% (223/223), 33.28 MiB | 35.53 MiB/s, done.
Resolving deltas: 100% (98/98), done.


In [ ]:
import os
from google.colab.patches import cv2_imshow
os.chdir('/content/keras_Realtime_Multi-Person_Pose_Estimation')
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **change 1** 


In [ ]:
import pandas as pd 
d=pd.read_csv("/content/sample_data/Shiv_bhuj.csv") 
del d['Unnamed: 0']

In [ ]:
d

,nose_X,nose_Y,neck_X,neck_Y,right_shoulder_X,right_shoulder_Y,right_elbow_X,right_elbow_Y,right_wrist_X,right_wrist_Y,left_shoulder_X,left_shoulder_Y,left_elbow_X,left_elbow_Y,left_wrist_X,left_wrist_Y,right_hip_X,right_hip_Y,right_knee_X,right_knee_Y,right_ankle_X,right_ankle_Y,left_hip_X,left_hip_Y,left_knee_X,left_knee_Y,left_ankle_X,left_ankle_Y,right_eye_X,right_eye_Y,left_eye_X,left_eye_Y,right_ear_X,right_ear_Y,left_ear_X,left_ear_Y,Activity
0,785.0,433.0,738,476,748,483,746,566,746,628,728,469,NaN,NaN,NaN,NaN,674,586,576,597,472.0,591.0,661.0,571.0,578.0,594.0,472.0,590.0,778.0,426.0,NaN,NaN,756,433,NaN,NaN,Bhuj
1,NaN,NaN,735,477,748,484,747,565,745,627,722,471,NaN,NaN,NaN,NaN,678,588,575,598,NaN,NaN,659.0,570.0,578.0,590.0,468.0,591.0,NaN,NaN,NaN,NaN,762,432,NaN,NaN,Bhuj
2,NaN,NaN,735,478,749,484,747,567,746,628,721,471,NaN,NaN,NaN,NaN,677,589,575,598,NaN,NaN,658.0,570.0,577.0,593.0,467.0,591.0,NaN,NaN,NaN,NaN,759,432,NaN,NaN,Bhuj
3,788.0,438.0,741,478,750,485,746,567,746,628,731,470,NaN,NaN,NaN,NaN,676,586,576,598,470.0,592.0,662.0,570.0,578.0,594.0,469.0,591.0,781.0,430.0,NaN,NaN,755,437,NaN,NaN,Bhuj
4,776.0,432.0,734,476,743,483,745,567,745,627,724,469,NaN,NaN,NaN,NaN,666,584,574,598,469.0,593.0,660.0,575.0,577.0,597.0,470.0,591.0,768.0,425.0,NaN,NaN,741,430,NaN,NaN,Bhuj
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,756.0,474.0,723,499,729,505,746,577,753,639,715,492,NaN,NaN,NaN,NaN,649,602,541,612,432.0,608.0,640.0,591.0,544.0,613.0,NaN,NaN,748.0,465.0,760.0,465.0,726,465,NaN,NaN,Bhuj
74,753.0,477.0,725,501,728,506,745,577,754,639,720,495,NaN,NaN,NaN,NaN,648,600,541,612,432.0,608.0,644.0,594.0,544.0,612.0,432.0,607.0,744.0,468.0,758.0,468.0,724,469,NaN,NaN,Bhuj
75,755.0,475.0,725,501,729,505,745,576,752,634,720,496,NaN,NaN,NaN,NaN,648,601,542,612,432.0,608.0,642.0,593.0,546.0,612.0,NaN,NaN,746.0,466.0,760.0,466.0,725,467,NaN,NaN,Bhuj
76,766.0,472.0,722,502,727,508,744,579,753,639,717,494,NaN,NaN,639.0,246.0,647,602,542,612,431.0,608.0,639.0,590.0,545.0,613.0,NaN,NaN,760.0,465.0,NaN,NaN,738,469,NaN,NaN,Bhuj


In [ ]:
pip install configobj

  Created wheel for configobj: filename=configobj-5.0.6-cp37-none-any.whl size=34547 sha256=85811c5fdca7e6aa762d1689e6393c8d1c55bcfe7c54754adc1d0a897fd98ddd
  Stored in directory: /root/.cache/pip/wheels/f1/e4/16/4981ca97c2d65106b49861e0b35e2660695be7219a2d351ee0
Successfully built configobj


In [ ]:
"""
0    -------------------------------nose
1    -------------------------------neck
2    -------------------------------right shoulder
3    -------------------------------right elbow
4    -------------------------------right wrist
5    -------------------------------left shoulder
6    -------------------------------left elbow
7    -------------------------------left wrist   
8    -------------------------------right hip
9    -------------------------------right knee
10   -------------------------------right ankle
11   -------------------------------left hip
12   -------------------------------left knee
13   -------------------------------left ankle
14   -------------------------------right eye
15   -------------------------------left eye
16   -------------------------------right ear
17   -------------------------------left ear    """

'\n0    -------------------------------nose\n1    -------------------------------neck\n2    -------------------------------right shoulder\n3    -------------------------------right elbow\n4    -------------------------------right wrist\n5    -------------------------------left shoulder\n6    -------------------------------left elbow\n7    -------------------------------left wrist   \n8    -------------------------------right hip\n9    -------------------------------right knee\n10   -------------------------------right ankle\n11   -------------------------------left hip\n12   -------------------------------left knee\n13   -------------------------------left ankle\n14   -------------------------------right eye\n15   -------------------------------left eye\n16   -------------------------------right ear\n17   -------------------------------left ear    '

In [ ]:
import argparse
import time
import cv2
from processing import extract_parts, draw
from config_reader import config_reader
from model.cmu_model import get_testing_model

In [ ]:
keras_weights_file = '/content/drive/MyDrive/keras_multi_pose/model.h5'
model = get_testing_model()
model.load_weights(keras_weights_file)
params, model_params = config_reader()
#scale_search = [1, .5, 1.5, 2]  # [.5, 1, 1.5, 2]
#scale_search = scale_search[0:4]
#params['scale_search'] = scale_search

In [ ]:
def extraction(input_image, params, model, model_params):
     tic = time.time()
     print('start processing...')
     body_parts, all_peaks, subset, candidate = extract_parts(input_image, params, model, model_params)
     canvas = draw(input_image, all_peaks, subset, candidate)
     toc = time.time()
     print('processing time is %.5f' % (toc - tic))
     #print(all_peaks)
     cv2_imshow(canvas)
     cv2.destroyAllWindows()
     return body_parts, all_peaks, subset, candidate

def points_extract(all_peaks):
 points = []
 for peak in all_peaks:
        try:
          m=peak[0][2:3]
          w_i=0
          for i in range(len(peak)):
             if(peak[i][2:3]>m):
                m=peak[i][2:3]
                w_i=i
          if(peak[w_i][2]>0.3):
             points.append((peak[w_i][:2]))
          else:
             points.append(("NA", "NA"))   
        except IndexError:
            points.append(("NA", "NA"))
 return points          


# **change 2 (two)**

In [ ]:
cap = cv2.VideoCapture('/content/drive/MyDrive/yoga vid collector/Veena_Bhu.mp4')
frameRate = 60
m=19
n=23
body_parts_arr=[]
all_peaks_arr=[]

In [ ]:
while(cap.isOpened()):
  frameId = cap.get(1)
  ret, frame = cap.read()
  if (ret != True):  
        break      
  if (frameId % frameRate == 0):
      print(frameId,"      ",frameId/frameRate)
      cv2_imshow(frame)
      if (m<=frameId/frameRate<=n):
          body_parts, all_peaks, subset, candidate=extraction(frame, params, model, model_params)
          body_parts_arr.append(body_parts)
          all_peaks_arr.append(all_peaks)
          print(body_parts)
          print(all_peaks)
  #Frame=frame    
cap.release()
print ("Done!")      

In [ ]:
#image_path = 'sample_images/image.jpg'
#input_image = cv2.imread(image_path)
#body_parts, all_peaks, subset, candidate=extraction(input_image, params, model, model_params)

In [ ]:
print(len(body_parts_arr),len(all_peaks_arr))

5 5


In [ ]:
points_arr=[]

# **change 3** 

In [ ]:
for i in range(5):
  points=points_extract(all_peaks_arr[i])
  points_arr.append(points)

In [ ]:
points_list_arr=[]

# **change 4**

In [ ]:
for j in range(5):
   points_list=[]
   for i in range(18):
      points_list.append(points_arr[j][i][0])
      points_list.append(points_arr[j][i][1])
   points_list.append("Bhuj")   
   points_list_arr.append(points_list)   

In [ ]:
len(points_list_arr[0])

37

In [ ]:
d

,nose_X,nose_Y,neck_X,neck_Y,right_shoulder_X,right_shoulder_Y,right_elbow_X,right_elbow_Y,right_wrist_X,right_wrist_Y,left_shoulder_X,left_shoulder_Y,left_elbow_X,left_elbow_Y,left_wrist_X,left_wrist_Y,right_hip_X,right_hip_Y,right_knee_X,right_knee_Y,right_ankle_X,right_ankle_Y,left_hip_X,left_hip_Y,left_knee_X,left_knee_Y,left_ankle_X,left_ankle_Y,right_eye_X,right_eye_Y,left_eye_X,left_eye_Y,right_ear_X,right_ear_Y,left_ear_X,left_ear_Y,Activity
0,785.0,433.0,738,476,748,483,746,566,746,628,728,469,NaN,NaN,NaN,NaN,674,586,576,597,472.0,591.0,661.0,571.0,578.0,594.0,472.0,590.0,778.0,426.0,NaN,NaN,756,433,NaN,NaN,Bhuj
1,NaN,NaN,735,477,748,484,747,565,745,627,722,471,NaN,NaN,NaN,NaN,678,588,575,598,NaN,NaN,659.0,570.0,578.0,590.0,468.0,591.0,NaN,NaN,NaN,NaN,762,432,NaN,NaN,Bhuj
2,NaN,NaN,735,478,749,484,747,567,746,628,721,471,NaN,NaN,NaN,NaN,677,589,575,598,NaN,NaN,658.0,570.0,577.0,593.0,467.0,591.0,NaN,NaN,NaN,NaN,759,432,NaN,NaN,Bhuj
3,788.0,438.0,741,478,750,485,746,567,746,628,731,470,NaN,NaN,NaN,NaN,676,586,576,598,470.0,592.0,662.0,570.0,578.0,594.0,469.0,591.0,781.0,430.0,NaN,NaN,755,437,NaN,NaN,Bhuj
4,776.0,432.0,734,476,743,483,745,567,745,627,724,469,NaN,NaN,NaN,NaN,666,584,574,598,469.0,593.0,660.0,575.0,577.0,597.0,470.0,591.0,768.0,425.0,NaN,NaN,741,430,NaN,NaN,Bhuj
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,756.0,474.0,723,499,729,505,746,577,753,639,715,492,NaN,NaN,NaN,NaN,649,602,541,612,432.0,608.0,640.0,591.0,544.0,613.0,NaN,NaN,748.0,465.0,760.0,465.0,726,465,NaN,NaN,Bhuj
74,753.0,477.0,725,501,728,506,745,577,754,639,720,495,NaN,NaN,NaN,NaN,648,600,541,612,432.0,608.0,644.0,594.0,544.0,612.0,432.0,607.0,744.0,468.0,758.0,468.0,724,469,NaN,NaN,Bhuj
75,755.0,475.0,725,501,729,505,745,576,752,634,720,496,NaN,NaN,NaN,NaN,648,601,542,612,432.0,608.0,642.0,593.0,546.0,612.0,NaN,NaN,746.0,466.0,760.0,466.0,725,467,NaN,NaN,Bhuj
76,766.0,472.0,722,502,727,508,744,579,753,639,717,494,NaN,NaN,639.0,246.0,647,602,542,612,431.0,608.0,639.0,590.0,545.0,613.0,NaN,NaN,760.0,465.0,NaN,NaN,738,469,NaN,NaN,Bhuj


# **change 5** 

In [ ]:
for i in range(5):
    d.loc[len(d)] = points_list_arr[i]

In [ ]:
d

,nose_X,nose_Y,neck_X,neck_Y,right_shoulder_X,right_shoulder_Y,right_elbow_X,right_elbow_Y,right_wrist_X,right_wrist_Y,left_shoulder_X,left_shoulder_Y,left_elbow_X,left_elbow_Y,left_wrist_X,left_wrist_Y,right_hip_X,right_hip_Y,right_knee_X,right_knee_Y,right_ankle_X,right_ankle_Y,left_hip_X,left_hip_Y,left_knee_X,left_knee_Y,left_ankle_X,left_ankle_Y,right_eye_X,right_eye_Y,left_eye_X,left_eye_Y,right_ear_X,right_ear_Y,left_ear_X,left_ear_Y,Activity
0,785,433,738,476,748,483,746,566,746,628,728,469,NaN,NaN,NaN,NaN,674,586,576,597,472,591,661.0,571.0,578,594,472,590,778.0,426.0,NaN,NaN,756,433,NaN,NaN,Bhuj
1,NaN,NaN,735,477,748,484,747,565,745,627,722,471,NaN,NaN,NaN,NaN,678,588,575,598,NaN,NaN,659.0,570.0,578,590,468,591,NaN,NaN,NaN,NaN,762,432,NaN,NaN,Bhuj
2,NaN,NaN,735,478,749,484,747,567,746,628,721,471,NaN,NaN,NaN,NaN,677,589,575,598,NaN,NaN,658.0,570.0,577,593,467,591,NaN,NaN,NaN,NaN,759,432,NaN,NaN,Bhuj
3,788,438,741,478,750,485,746,567,746,628,731,470,NaN,NaN,NaN,NaN,676,586,576,598,470,592,662.0,570.0,578,594,469,591,781.0,430.0,NaN,NaN,755,437,NaN,NaN,Bhuj
4,776,432,734,476,743,483,745,567,745,627,724,469,NaN,NaN,NaN,NaN,666,584,574,598,469,593,660.0,575.0,577,597,470,591,768.0,425.0,NaN,NaN,741,430,NaN,NaN,Bhuj
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,NA,NA,679,530,688,536,703,597,720,637,669,524,NA,NA,NA,NA,624,618,NA,NA,NA,NA,603.0,599.0,NA,NA,NA,NA,711.0,487.0,NA,NA,702,494,NA,NA,Bhuj
79,714,487,679,530,687,536,703,597,720,638,671,524,NA,NA,NA,NA,619,618,NA,NA,NA,NA,605.0,601.0,NA,NA,NA,NA,706.0,482.0,NA,NA,685,495,NA,NA,Bhuj
80,709,489,682,530,689,536,701,594,721,639,675,522,NA,NA,NA,NA,622,615,NA,NA,NA,NA,610.0,602.0,NA,NA,NA,NA,698.0,484.0,710,482,676,496,NA,NA,Bhuj
81,713,493,683,529,691,537,703,594,720,638,676,522,NA,NA,NA,NA,622,617,NA,NA,NA,NA,608.0,602.0,NA,NA,NA,NA,702.0,487.0,714,485,678,496,NA,NA,Bhuj


In [ ]:
d

,nose_X,nose_Y,neck_X,neck_Y,right_shoulder_X,right_shoulder_Y,right_elbow_X,right_elbow_Y,right_wrist_X,right_wrist_Y,left_shoulder_X,left_shoulder_Y,left_elbow_X,left_elbow_Y,left_wrist_X,left_wrist_Y,right_hip_X,right_hip_Y,right_knee_X,right_knee_Y,right_ankle_X,right_ankle_Y,left_hip_X,left_hip_Y,left_knee_X,left_knee_Y,left_ankle_X,left_ankle_Y,right_eye_X,right_eye_Y,left_eye_X,left_eye_Y,right_ear_X,right_ear_Y,left_ear_X,left_ear_Y,Activity
0,785,433,738,476,748,483,746,566,746,628,728,469,NA,NA,NA,NA,674,586,576,597,472,591,661,571,578,594,472,590,778,426,NA,NA,756,433,NA,NA,Bhuj
1,NA,NA,735,477,748,484,747,565,745,627,722,471,NA,NA,NA,NA,678,588,575,598,NA,NA,659,570,578,590,468,591,NA,NA,NA,NA,762,432,NA,NA,Bhuj
2,NA,NA,735,478,749,484,747,567,746,628,721,471,NA,NA,NA,NA,677,589,575,598,NA,NA,658,570,577,593,467,591,NA,NA,NA,NA,759,432,NA,NA,Bhuj
3,788,438,741,478,750,485,746,567,746,628,731,470,NA,NA,NA,NA,676,586,576,598,470,592,662,570,578,594,469,591,781,430,NA,NA,755,437,NA,NA,Bhuj
4,776,432,734,476,743,483,745,567,745,627,724,469,NA,NA,NA,NA,666,584,574,598,469,593,660,575,577,597,470,591,768,425,NA,NA,741,430,NA,NA,Bhuj
5,773,427,734,476,746,483,746,566,746,628,722,469,NA,NA,NA,NA,671,587,574,598,NA,NA,658,572,577,595,469,590,769,422,NA,NA,758,431,NA,NA,Bhuj
6,779,435,736,478,746,484,745,567,746,629,725,471,NA,NA,NA,NA,669,586,574,598,NA,NA,659,573,577,595,468,592,772,428,NA,NA,749,436,NA,NA,Bhuj


# **change 6**

In [ ]:
d.to_csv('/content/sample_data/Veena_Bhu.csv')